## 출력 파서
LLM의 출력값을 구조화된 형식으로 변환하고 답변에서 우리가 원하는 정보만 뽑아낼 때 유용하게 사용되는 도구    
    
    
특징
- 다양성 : 다양한 변환 방식에 대응
- 스트리밍 지원 → 실시간 데이터 처리 가능
- 확장성 : 확장 가능한 인터페이스 제공
    
    
장점
- 구조화 : LLM의 자유 형식 텍스트를 구조화된 데이터로 변환해 정보를 체계적으로 관리 가능
- 일관성 : 출력 형식을 일관되게 유지하여 후속 처리나 데이터 조회가 쉽고 효율적
- 유연성 : JSON, 리스트, 딕셔너리 등 다양한 출력 형식으로 변환 가능하여 유연성 제공

### `PydanticOutputParser`
- 언어 모델의 출력을 특정 데이터 모델에 맞게 변환
- `get_format_instructions()` : 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침 제공 
- `parse()` : 언어 모델의 출력을 받아 이를 특정 구조로 분석하고 변환

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os, sys

sys.path.append(os.path.abspath(".."))
load_dotenv()

True

In [6]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [ ]:
# 출력 파서 쓰기 전

from itertools import chain
from langchain_core.prompts import PromptTemplate
from app import stream_response

prompt = PromptTemplate.from_template("다음의 이메일 내용 중 중요한 내용을 추출해 주세요.\n\n{email_conversation}")

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

output = stream_response(answer, return_output=True)

이메일의 중요한 내용은 다음과 같습니다:

1. 김철수 상무는 바이크코퍼레이션 소속이며, 이은채 대리에게 이메일을 보냈습니다.
2. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 활동하는 기업입니다.
3. 김철수 상무는 "ZENESIS" 자전거 모델에 대한 상세한 브로슈어를 요청하고 있습니다. 특히 기술 사양, 배터리 성능, 디자인 측면의 정보가 필요합니다.
4. 협력 가능성을 논의하기 위해 1월 15일 화요일 오전 10시에 미팅을 제안했습니다. 미팅 장소는 귀사 사무실입니다.

In [ ]:
# Pydantic 스타일로 이메일 정보 파싱

class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [ ]:
# llm이 출력해야 할 정보의 형식 
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [10]:
# 프롬프트 템플릿 작성
prompt = PromptTemplate.from_template(
    """
    You are a helpful assistant. Please answer the following questions in KOREAN.
    
    QUESTION:
    {question}
    
    EMAIL CONVERSATION:
    {email_conversation}
    
    FORMAT:
    {format}
    """
)

In [20]:
# format 변수만 채워두기
prompt = prompt.partial(format=parser.get_format_instructions())
prompt

PromptTemplate(input_variables=['email_conversation', 'question'], input_types={}, partial_variables={'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required

In [ ]:
chain = prompt | llm

response = chain.stream({  # 나머지 변수도 채우기 
    "email_conversation": email_conversation,
    "question": "이메일 내용 중 주요 내용을 추출해주세요."
})
output = stream_response(response, return_output=True)

```json
{
    "person": "김철수",
    "email": "chulsoo.kim@bikecorporation.me",
    "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
    "summary": "김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 위해 ZENESIS 모델의 상세한 브로슈어를 요청하고, 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다.",
    "date": "1월 15일 오전 10시"
}
```

In [ ]:
# LLM 츌력 결과 파싱
structured_output = parser.parse(output)
structured_output

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 위해 ZENESIS 모델의 상세한 브로슈어를 요청하고, 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다.', date='1월 15일 오전 10시')

In [16]:
structured_output.person

'김철수'

In [ ]:
# 출력 파서 자체를 체인에 추가하여 파싱한 답변을 출력
chain = prompt | llm | parser

response = chain.invoke({
    "email_conversation": email_conversation,
    "question": "이메일 내용 중 주요 내용을 추출해주세요."
})
response

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 위해 ZENESIS 모델의 상세한 브로슈어를 요청하고, 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다.', date='1월 15일 오전 10시')

### `with_structured_output()` 바인딩

In [22]:
llm_structured = ChatOpenAI(temperature=0, model_name="gpt-4o").with_structured_output(EmailSummary)

answer = llm_structured.invoke(email_conversation)
answer

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무는 이은채 대리님에게 바이크코퍼레이션과 테디인터내셔널 간의 협력 가능성을 논의하기 위해 이메일을 보냈습니다. 그는 테디인터내셔널의 신규 자전거 모델 "ZENESIS"에 대한 관심을 표명하며, 상세한 브로슈어를 요청했습니다. 특히 기술 사양, 배터리 성능, 디자인 측면에 대한 정보를 필요로 하며, 이를 바탕으로 유통 전략과 마케팅 계획을 구체화하고자 합니다. 또한, 협력 논의를 위해 1월 15일 화요일 오전 10시에 미팅을 제안했습니다.', date='1월 15일 화요일 오전 10시')

In [23]:
answer.person

'김철수'

### 쉼표로 구분된 리스트 출력 파서

In [24]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [25]:
# 프롬프트 템플릿
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"], 
    partial_variables={"format_instructions": format_instructions},  # 변수 미리 채우기
)

# 체인
chain = prompt | llm | output_parser

# 답변
answer = chain.invoke({"subject": "대한민국 관광명소"})
answer

['경복궁', '제주도', '부산 해운대', '설악산', '경주 불국사']

In [31]:
# 스트리밍 출력 : 토큰 단위로 별개의 리스트가 됨
for s in chain.stream({"subject": "대한민국 관광명소"}):
    print(s)

['경복궁']
['제주도']
['부산 해운대']
['경주 불국사']
['설악산 국립공원']


### 구조화된 출력 파서

In [33]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

# 사용자의 질문에 대한 답변과 사용된 소스에 대한 설명을 포함하는 응답 스키마 정의
# 로컬 모델 : 인텔리전스가 부족할 수 있기 때문에 description을 영어로 작성하는걸 추천
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"), 
    ResponseSchema(name="source", description="사용자의 질문에 답하기 위해 사용된 '출처', '웹사이트 주소'이어야 합니다.")
]

In [ ]:
# 출력 형식에 대한 response_schemas의 지침
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

print(output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 사용자의 질문에 대한 답변
	"source": string  // 사용자의 질문에 답하기 위해 사용된 '출처', '웹사이트 주소'이어야 합니다.
}
```


In [36]:
# 출력 형식 지시사항을 파싱
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿 설정
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(temperature=0)
chain = prompt | model | output_parser
chain.invoke({"question": "대한민국의 수도는 어디인가요?"})

{'answer': '서울', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%9C%EC%9A%B8'}

### JSON 형식 출력 파서
LLM이 데이터를 처리하여 JSON을 생성하기 위해서는 충분한 용량이 필요    
⇒ 용량이 작은 모델에서는 `JsonOutputParser` 사용 시 오류가 발생할 수 있음

In [40]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

model = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [38]:
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [39]:
question = "지구 온난화의 심각성에 대해 알려주세요."

parser = JsonOutputParser(pydantic_object=Topic)
print(parser.get_format_instructions())  # 예시, 스키마를 담은 지침

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "주제에 대한 간결한 설명", "title": "Description", "type": "string"}, "hashtags": {"description": "해시태그 형식의 키워드(2개 이상)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [41]:
# 템플릿 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절한 AI 어시스턴트입니다. 질문에 간결하게 답변하세요."), 
    ("user", "#Format: {format_instructions}\n\n#Question: {question}")
])
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser
answer = chain.invoke({"question": question})
answer

{'description': '지구 온난화는 지구의 평균 기온이 상승하는 현상으로, 주로 인간의 활동에 의해 발생하는 온실가스 배출이 주요 원인입니다. 이는 극지방의 빙하가 녹고 해수면이 상승하며, 기후 패턴이 변화하여 극단적인 기상 현상이 증가하는 등 심각한 환경적 영향을 미칩니다.',
 'hashtags': '#지구온난화 #기후변화 #환경문제'}

In [42]:
answer['description']

'지구 온난화는 지구의 평균 기온이 상승하는 현상으로, 주로 인간의 활동에 의해 발생하는 온실가스 배출이 주요 원인입니다. 이는 극지방의 빙하가 녹고 해수면이 상승하며, 기후 패턴이 변화하여 극단적인 기상 현상이 증가하는 등 심각한 환경적 영향을 미칩니다.'

### Pandas 데이터프레임 출력 파서

In [53]:
import pprint
from typing import Any, Dict

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser

In [54]:
model = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [55]:
# 파서 출력을 사전 형식으로 변환하고 출력 형식을 지정 => 출력을 깔끔하게 정리
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    for key in parser_output.keys():
        parser_output[key] = parser_output[key].to_dict()
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

In [56]:
df = pd.read_csv("../data/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [57]:
parser = PandasDataFrameOutputParser(dataframe=df)

print(parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [58]:
df_query = "Age column을 조회해주세요."

prompt = PromptTemplate(
    template="Answer the user query. \n{format_instructions}\n{question}\n", 
    input_variables=["question"], 
    partial_variables={"format_instructions": parser.get_format_instructions()}, 
)

chain = prompt | model | parser

parser_output = chain.invoke({"question": df_query})

format_parser_output(parser_output)

{'Age': {0: 22.0,
         1: 38.0,
         2: 26.0,
         3: 35.0,
         4: 35.0,
         5: nan,
         6: 54.0,
         7: 2.0,
         8: 27.0,
         9: 14.0,
         10: 4.0,
         11: 58.0,
         12: 20.0,
         13: 39.0,
         14: 14.0,
         15: 55.0,
         16: 2.0,
         17: nan,
         18: 31.0,
         19: nan}}


In [60]:
# 데이터프레임 첫 번째 행 조회
df_query = "Retrieve the first row."

parser_output = chain.invoke({"question": df_query})

format_parser_output(parser_output)

{'0': {'Age': 22.0,
       'Cabin': nan,
       'Embarked': 'S',
       'Fare': 7.25,
       'Name': 'Braund, '
               'Mr. '
               'Owen '
               'Harris',
       'Parch': 0,
       'PassengerId': 1,
       'Pclass': 3,
       'Sex': 'male',
       'SibSp': 1,
       'Survived': 0,
       'Ticket': 'A/5 '
                 '21171'}}


In [62]:
# 0~4번재 행까지 나이 열의 평균 계산
df_query = "Retrieve the average of the Ages from row 0 to 4."

parser_output = chain.invoke({"question": df_query})

print(parser_output)

{'mean': np.float64(31.2)}


In [63]:
# 요금(Fare) 평균
df_query = "Calculate average 'Fare' rate."

parser_output = chain.invoke({"question": df_query})

print(parser_output)

{'mean': np.float64(22.19937)}


### 날짜 형식 출력 파서
LLM의 출력을 `datetime` 형식으로 파싱

In [64]:
from langchain.output_parsers import DatetimeOutputParser

output_parser = DatetimeOutputParser()
output_parser.format = "%Y-%m-%d"

print(output_parser.get_format_instructions())

Write a datetime string that matches the following pattern: '%Y-%m-%d'.

Examples: 2025-10-10, 2024-10-10, 2025-10-09

Return ONLY this string, no other words!


In [65]:
template = """Answer the users question:

#Format Instructions:
{format_instructions}

#Question:
{question}

#Answer:
"""

prompt = PromptTemplate.from_template(
    template, 
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%d'.\n\nExamples: 2025-10-10, 2024-10-10, 2025-10-09\n\nReturn ONLY this string, no other words!"}, template='Answer the users question:\n\n#Format Instructions:\n{format_instructions}\n\n#Question:\n{question}\n\n#Answer:\n')

In [69]:
chain = prompt | ChatOpenAI() | output_parser

output = chain.invoke({"question": "Google이 창업한 연도"})
output

datetime.datetime(1998, 9, 4, 0, 0)

In [70]:
output.strftime("%Y-%m-%d")

'1998-09-04'

### 열거형 출력 파서
LLM의 출력을 미리 정의된 열거형 값 중 하나로 파싱하는 도구

In [71]:
from enum import Enum
from langchain.output_parsers import EnumOutputParser

class Colors(Enum):
    RED = "빨간색"
    GREEN = "초록색"
    BLUE = "파란색"

In [72]:
parser = EnumOutputParser(enum=Colors)
parser.get_format_instructions()

'Select one of the following options: 빨간색, 초록색, 파란색'

In [73]:
prompt = PromptTemplate.from_template(
    """다음의 물체는 어떤 색깔인가요?
    
    Object: {object}
    
    Instructions: {instructions}"""
).partial(instructions=parser.get_format_instructions())

chain = prompt | ChatOpenAI() | parser

response = chain.invoke({"object": "하늘"})
print(response)

Colors.BLUE


In [74]:
type(response)

<enum 'Colors'>

In [75]:
response.value

'파란색'